# Titanic - Preprocessing Data


## Preparation

### Import necessary libraries

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Load train.csv file

In [ ]:
path_dir = os.path.join("..", "..", "data", "raw")
titanic = pd.read_csv(os.path.join(path_dir, "train.csv"))
df = titanic.copy()
test_dataset = pd.read_csv(os.path.join(path_dir, "test.csv"))
df_test = test_dataset.copy()
print("Successfully load training data.")

Successfully load training data.


## Handle missing value

We need to find solutions to handle missing value.
From the output detecting missing value, here's the proportion of missing value for `Age`, `Cabin`, and `Embarked` in train dataset:

- Age: 19.87%
- Cabin: 77.10%
- Embarked: 0.22%

For test dataset, here's the proportion of missing value for `Age`, `Cabin`, and `Fare`:
- Age: 20.57%
- Cabin: 78.23%
- Fare: 0.24%

For `Cabin` features, although the missing value percentage are pretty high, it could be a good information to show passenger's room level, which is a really crucial detail related to survivability. Furthermore, we can base on that information to know if the passengers stayed at the front part (bow) of the Titanic, which is the part of the ship that sank first.

For `Age` features, it could related to the survivability, as children and elderly usually be prioritized.

For `Embarked` features, it could related to passenger's room level (passengers go on the ship first might have room that are in higher level).

Because of that, I think that we should not remove these features. We need to handle value instead.

For testing dataset, we also have missing value in `Fare` feature. We will handle it.

### Handle missing age value

Honorific title inside passengers' name could tell us a little about their age.

At first, we splited value `Age` into 5 main groups: Mr, Mrs, Master, Miss, and other in order to not only impute missing values but also capture potential age-related patterns.

But after looking at survivability rate of each group, we decided not to complicate things and just impute missing value with median age of the whole dataset. Except for `Master` group, we will impute missing value with median age of this group as title "Master" is only for people under 12.

In [10]:
master = df['Name'].str.contains(r',\s*Master.', regex=True)
df_master = df[master].copy()

mean_age = df['Age'].mean()
mean_age_master = df_master['Age'].mean()

df_master['Age'] = df_master['Age'].fillna(mean_age_master)
df[master] = df_master

df['Age'] = df['Age'].fillna(mean_age)
df_test['Age'] = df_test['Age'].fillna(mean_age)

### Handle missing values from Cabin, Embarked (train dataset) and Fare (test dataset)

Perhaps the only features that we could use to guess people's cabin is Ticket number, but since there are no ticket number system, we could not conclude anything from that.

At first, I thought it would be possible to fill missing `Cabin` value by looking at passengers' ticket class, fare, and embarked point, but after some research, I found out that there is no clear relationship between these features and `Cabin` value.

Therefore, I will just remove the `Cabin` feature, as it has too many missing values and we could not find a way to fill them.

As for Embarked value, I will choose "S" to fill that, as the primary embarked point was Southampton (Titanic trip started from there)

As for Fare value in testing dataset, I will fill it with median of Fare value.

In [11]:
df = df.drop(['Cabin'], axis=1)
df_test = df_test.drop(['Cabin'], axis=1)

df['Embarked'] = df['Embarked'].fillna("S")
df_test['Embarked'] = df_test['Embarked'].fillna("S")

fare_mean = df['Fare'].mean()
df_test['Fare'] = df_test['Fare'].fillna(fare_mean)

In [ ]:
print("===== Quantity of missing values =====")
df[['Age', 'Embarked']].isnull().sum()
df_test[['Fare']].isnull().sum()
print()
print("===== Percentage of missing values =====")
df[['Age', 'Embarked']].isnull().sum() / len(df) * 100
df_test[['Fare']].isnull().sum() / len(df_test) * 100

===== Percentage of missing values =====


Age         0.0
Embarked    0.0
dtype: float64

## Detecting outliers

This section will mainly focus on addressing outlier values in numeric features.

Based on the outputs from the boxplot from EDA Analysis, we could see that `Age`, `Fare`, `SibSp`, and `Parch` features have outlier values. However, since `SibSp` and `Parch` features are not continuous (both features have a limited number of unique values), we will not handle outlier values in these two features, as it could affect negatively on the model.

To handle outlier values, IQR method is used to find the lower bound and upper bound, then replace outlier values with values within these bounds using techniques such as capping or imputation.

In [13]:
num_cols = ['Age', 'Fare']
def detect_outliers_iqr(df, col, factor=1.5):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = max(Q1 - factor * IQR, 0)  # Age cannot be negative
    upper_bound = Q3 + factor * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    # print(f'{col} - Outliers (IQR): {len(outliers)}, Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}')
    return outliers, lower_bound, upper_bound


for col in num_cols:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(df, col)
    print(f'Total outliers in {col}: {len(outliers)}')

Total outliers in Age: 66
Total outliers in Fare: 116


Base on the outcome, we could see that:
- Total outliers in Age: 66
- Total outliers in Fare: 116

The number of outliers in `Fare` is quite high, we will have to handle it carefully in the next part.

## Save file

We will save train and test files to use for the future

In [ ]:
df.to_csv(os.path.join(path_dir, "preprocessed", "preprocessed_train.csv"))
df_test.to_csv(os.path.join(path_dir, "preprocessed", "preprocessed_test.csv"))
print("Successfully saved preprocessed data.")

Successfully saved preprocessed data.


# The end